In [6]:
%load_ext autoreload
%autoreload 2

import os
import sys
import importlib
import ipywidgets as widgets
from pathlib import Path

module_path='preprocessing'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='utils'
if module_path not in sys.path:
    sys.path.append(module_path)

root_dir = os.path.dirname(os.path.abspath('mainPipeline.ipynb'))

import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

import feature_selection_hosp
from feature_selection_hosp import *

## 1. DATA EXTRACTION
Please run below cell to select option for cohort selection.
The cohort will be svaed in **./data/cohort/**

In [2]:
version = 'Version 2'
radio_input4 = 'Phenotype'

### Refining Cohort and Prediction Task Definition

Based on your current selection following block will provide option to further refine prediction task and cohort associated with it:

- First you will refine the prediction task choosing from following options -
    - **Phenotype Prediction** - You can select from four major chronic diseases to predict its future outcome

        - Heart failure
        - CAD (Coronary Artery Disease)
        - CKD (Chronic Kidney Disease)
        - COPD (Chronic obstructive pulmonary disease)

- Second, you will choode whether to perfom above task using ICU or non-ICU admissions data

- Third, you can refine the refine the cohort selection for any of the above choosen prediction tasks by including the admission samples admitted with particular chronic disease - 
    - Heart failure
    - CAD (Coronary Artery Disease)
    - CKD (Chronic Kidney Disease)
    - COPD (Chronic obstructive pulmonary disease)
    
print("**Please run below cell to extract the cohort for selected options**")

In [3]:
radio_input2 = widgets.RadioButtons(options=['Heart Failure in 30 days','CAD in 30 days','CKD in 30 days','COPD in 30 days'],value='Heart Failure in 30 days')
display(radio_input2)

print("Extract Data")
radio_input1 = 'Non-ICU'

print("Please select if you want to perform choosen prediction task for a specific disease.")
radio_input3 = widgets.RadioButtons(options=['No Disease Filter','Heart Failure','CKD','CAD','COPD'],value='No Disease Filter')
display(radio_input3)

RadioButtons(options=('Heart Failure in 30 days', 'CAD in 30 days', 'CKD in 30 days', 'COPD in 30 days'), valu…

Extract Data
Please select if you want to perform choosen prediction task for a specific disease.


RadioButtons(options=('No Disease Filter', 'Heart Failure', 'CKD', 'CAD', 'COPD'), value='No Disease Filter')

In [4]:
disease_label=""
time=0

if radio_input2.value=='Heart Failure in 30 days':
    label='Readmission'
    time=30
    disease_label='I50'
elif radio_input2.value=='CAD in 30 days':
    label='Readmission'
    time=30
    disease_label='I25'
elif radio_input2.value=='CKD in 30 days':
    label='Readmission'
    time=30
    disease_label='N18'
elif radio_input2.value=='COPD in 30 days':
    label='Readmission'
    time=30
    disease_label='J44'
    
data_icu=False
data_mort=label=="Mortality"
data_admn=label=='Readmission'
data_los=label=='Length of Stay'
        
if (radio_input3.value=="Heart Failure"):
    icd_code='I50'
elif (radio_input3.value=="CKD"):
    icd_code='N18'
elif (radio_input3.value=="COPD"):
    icd_code='J44'
elif (radio_input3.value=="CAD"):
    icd_code='I25'
else:
    icd_code='No Disease Filter'

version_path="mimiciv/2.2"
cohort_output = day_intervals_cohort_v2.extract_data(radio_input1,label,time,icd_code, root_dir,disease_label)

===========MIMIC-IV v2.0============
EXTRACTING FOR: | NON-ICU | READMISSION DUE TO I50 | 30 | 
[ READMISSION DUE TO I50 ]


100%|██████████| 22581/22581 [03:33<00:00, 105.73it/s]


[ READMISSION LABELS FINISHED ]
[ COHORT SUCCESSFULLY SAVED ]
[ SUMMARY SUCCESSFULLY SAVED ]
Readmission FOR Non-ICU DATA
# Admission Records: 58079
# Patients: 22581
# Positive cases: 11867
# Negative cases: 46212


## 2. FEATURE SELECTION
Features available for Non-ICU data -
- Diagnosis (https://mimic.mit.edu/docs/iv/modules/hosp/diagnoses_icd/)
- Procedures (https://mimic.mit.edu/docs/iv/modules/hosp/procedures_icd/)
- Medications (https://mimic.mit.edu/docs/iv/modules/hosp/prescriptions/)
- Lab Events (https://mimic.mit.edu/docs/iv/modules/hosp/labevents/)

All features will be saved in **./data/features/**

**Please run below cell to select features**

In [8]:
diag_flag=True
lab_flag=True
proc_flag=True
med_flag=True
feature_nonicu(cohort_output, version_path,diag_flag,lab_flag,proc_flag,med_flag)

[EXTRACTING DIAGNOSIS DATA]


100%|██████████| 5011/5011 [00:11<00:00, 429.98it/s]


# unique ICD-9 codes 5011
# unique ICD-10 codes 7882
# unique ICD-10 codes (After converting ICD-9 to ICD-10) 8195
# unique ICD-10 codes (After clinical gruping ICD-10 codes) 1346
# Admissions:   58079
[SUCCESSFULLY SAVED DIAGNOSIS DATA]
[EXTRACTING PROCEDURES DATA]
# Unique ICD9 Procedures:   1331
# Unique ICD10 Procedures:  3757

Value counts of each ICD version:
 9     65570
10    40698
Name: icd_version, dtype: int64
# Admissions:   32486
Total number of rows:  106268
[SUCCESSFULLY SAVED PROCEDURES DATA]
[EXTRACTING MEDICATIONS DATA]
Number of unique type of drug:  1790
Number of unique type of drug (after grouping to use Non propietary names):  919
Total number of rows:  1843549
# Admissions:   56606
[SUCCESSFULLY SAVED MEDICATIONS DATA]
[EXTRACTING LABS DATA]


0it [00:00, ?it/s]/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
1it [07:15, 435.24s/it]/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
2it [15:18, 463.47s/it]/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/pand

# Itemid:  537
# Admissions:  57662
Total number of rows:  14556183
[SUCCESSFULLY SAVED LABS DATA]


## 3. CLINICAL GROUPING
Below you will have option to clinically group diagnosis and medications.
Grouping medical codes will reduce dimensional space of features.

Default options selected below will group medical codes to reduce feature dimension space.

**Please run below cell to select preprocessing for diferent features**

In [9]:
if diag_flag:
    print("Do you want to group ICD 10 DIAG codes ?")
    radio_input4 = widgets.RadioButtons(options=['Keep both ICD-9 and ICD-10 codes','Convert ICD-9 to ICD-10 codes','Convert ICD-9 to ICD-10 and group ICD-10 codes'],value='Convert ICD-9 to ICD-10 and group ICD-10 codes',layout={'width': '100%'})
    display(radio_input4)     
if med_flag:
    print("Do you want to group Medication codes to use Non propietary names?")
    radio_input5 = widgets.RadioButtons(options=['Yes','No'],value='Yes',layout={'width': '100%'})
    display(radio_input5)
if proc_flag:
    print("Which ICD codes for Procedures you want to keep in data?")
    radio_input6 = widgets.RadioButtons(options=['ICD-9 and ICD-10','ICD-10'],value='ICD-10',layout={'width': '100%'})
    display(radio_input6)
print("**Please run below cell to perform feature preprocessing**")

Do you want to group ICD 10 DIAG codes ?


RadioButtons(index=2, layout=Layout(width='100%'), options=('Keep both ICD-9 and ICD-10 codes', 'Convert ICD-9…

Do you want to group Medication codes to use Non propietary names?


RadioButtons(layout=Layout(width='100%'), options=('Yes', 'No'), value='Yes')

Which ICD codes for Procedures you want to keep in data?


RadioButtons(index=1, layout=Layout(width='100%'), options=('ICD-9 and ICD-10', 'ICD-10'), value='ICD-10')

**Please run below cell to perform feature preprocessing**


In [10]:
group_diag=False
group_med=False
group_proc=False
if diag_flag:
    group_diag=radio_input4.value
if med_flag:
    group_med=radio_input5.value
if proc_flag:
    group_proc=radio_input6.value
preprocess_features_hosp(cohort_output, diag_flag,proc_flag,med_flag,False,group_diag,group_med,group_proc,False,False,0,0)

[PROCESSING DIAGNOSIS DATA]
Total number of rows 1016267
[SUCCESSFULLY SAVED DIAGNOSIS DATA]
[PROCESSING MEDICATIONS DATA]


/tmp/ipykernel_2501519/953078655.py:10: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  preprocess_features_hosp(cohort_output, diag_flag,proc_flag,med_flag,False,group_diag,group_med,group_proc,False,False,0,0)


Total number of rows 1842600
[SUCCESSFULLY SAVED MEDICATIONS DATA]
[PROCESSING PROCEDURES DATA]
Total number of rows 40698
[SUCCESSFULLY SAVED PROCEDURES DATA]


### 4. SUMMARY OF FEATURES

This step will generate summary of all features extracted so far.<br>
It will save summary files in **./data/summary/**<br>
- These files provide summary about **mean frequency** of medical codes per admission.<br>
- It also provides **total occurrence count** of each medical code.<br>
- For labs and chart events it will also provide <br>**missing %** which tells how many rows for a certain medical code has missing value.

Please use this information to further refine your cohort by selecting <br>which medical codes in each feature you want to keep and <br>which codes you would like to remove for downstream analysis tasks.

**Please run below cell to generate summary files**

In [11]:
generate_summary_hosp(diag_flag,proc_flag,med_flag,lab_flag)

[GENERATING FEATURE SUMMARY]


/tmp/ipykernel_2501519/1553721925.py:1: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_summary_hosp(diag_flag,proc_flag,med_flag,lab_flag)
2it [00:15,  7.93s/it]


[SUCCESSFULLY SAVED FEATURE SUMMARY]


## 5. Feature Selection

based on the files generated in previous step and other infromation gathered by you,<br>
Please select which medical codes you want to include in this study.

Please run below cell to to select options for which features you want to perform feature selection.

- Select **Yes** if you want to select a subset of medical codes for that feature and<br> **edit** the corresponding feature file for it.
- Select **No** if you want to keep all the codes in a feature.

In [12]:
if diag_flag:
    print("Do you want to do Feature Selection for Diagnosis \n (If yes, please edit list of codes in ./data/summary/diag_features.csv)")
    radio_input4 = widgets.RadioButtons(options=['Yes','No'],value='No')
    display(radio_input4)         
if med_flag:
    print("Do you want to do Feature Selection for Medication \n (If yes, please edit list of codes in ./data/summary/med_features.csv)")
    radio_input5 = widgets.RadioButtons(options=['Yes','No'],value='No')
    display(radio_input5)   
if proc_flag:
    print("Do you want to do Feature Selection for Procedures \n (If yes, please edit list of codes in ./data/summary/proc_features.csv)")
    radio_input6 = widgets.RadioButtons(options=['Yes','No'],value='No')
    display(radio_input6)   
if lab_flag:
    print("Do you want to do Feature Selection for Labs \n (If yes, please edit list of codes in ./data/summary/lab_features.csv)")
    radio_input7 = widgets.RadioButtons(options=['Yes','No'],value='No')
    display(radio_input7)   
print("**Please run below cell to perform feature selection**")

Do you want to do Feature Selection for Diagnosis 
 (If yes, please edit list of codes in ./data/summary/diag_features.csv)


RadioButtons(index=1, options=('Yes', 'No'), value='No')

Do you want to do Feature Selection for Medication 
 (If yes, please edit list of codes in ./data/summary/med_features.csv)


RadioButtons(index=1, options=('Yes', 'No'), value='No')

Do you want to do Feature Selection for Procedures 
 (If yes, please edit list of codes in ./data/summary/proc_features.csv)


RadioButtons(index=1, options=('Yes', 'No'), value='No')

Do you want to do Feature Selection for Labs 
 (If yes, please edit list of codes in ./data/summary/lab_features.csv)


RadioButtons(index=1, options=('Yes', 'No'), value='No')

**Please run below cell to perform feature selection**


In [13]:
select_diag=False
select_med=False
select_proc=False
select_lab=False
select_out=False
select_chart=False

if diag_flag:
    select_diag=radio_input4.value == 'Yes'
if med_flag:
    select_med=radio_input5.value == 'Yes'
if proc_flag:
    select_proc=radio_input6.value == 'Yes'
if lab_flag:
    select_lab=radio_input7.value == 'Yes'
features_selection_hosp(cohort_output, diag_flag,proc_flag,med_flag,lab_flag,select_diag,select_med,select_proc,select_lab)

## 6. CLEANING OF FEATURES
Below you will have option to to clean lab and chart events by performing outlier removal and unit conversion.

Outlier removal is performed to remove values higher than selected **right threshold** percentile and lower than selected **left threshold** percentile among all values for each itemid. 

**Please run below cell to select preprocessing for diferent features**

In [14]:
if lab_flag:
    print("Outlier removal in values of lab events ?")
    layout = widgets.Layout(width='100%', height='40px') #set width and height

    radio_input7 = widgets.RadioButtons(options=['No outlier detection','Impute Outlier (default:98)','Remove outliers (default:98)'],value='No outlier detection',layout=layout)
    display(radio_input7)
    outlier=widgets.IntSlider(
    value=98,
    min=90,
    max=99,
    step=1,
    disabled=False,layout={'width': '100%'}
    )
    left_outlier=widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    disabled=False,layout={'width': '100%'}
    )
    #display(oulier)
    display(widgets.HBox([widgets.Label('Right Outlier Threshold',layout={'width': '150px'}), outlier]))
    display(widgets.HBox([widgets.Label('Left Outlier Threshold',layout={'width': '150px'}), left_outlier]))
print("**Please run below cell to perform feature preprocessing**")

Outlier removal in values of lab events ?


RadioButtons(layout=Layout(height='40px', width='100%'), options=('No outlier detection', 'Impute Outlier (def…

**Please run below cell to perform feature preprocessing**


In [15]:
thresh=0
if lab_flag:
    clean_lab=radio_input7.value!='No outlier detection'
    impute_outlier=radio_input7.value=='Impute Outlier (default:98)'
    thresh=outlier.value
    left_thresh=left_outlier.value
preprocess_features_hosp(cohort_output, False,False,False,lab_flag,False,False,False,clean_lab,impute_outlier,thresh,left_thresh)

## 7. Add diag names and lab names

In [99]:
read_dir = './data/features/'
save_dir = './processed/'

### cohort

In [168]:
cohort = pd.read_csv('./data/cohort/cohort_non-icu_readmission_30_I50.csv.gz', compression='gzip')
cohort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58079 entries, 0 to 58078
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  58079 non-null  float64
 1   hadm_id     58079 non-null  float64
 2   admittime   58079 non-null  object 
 3   dischtime   58079 non-null  object 
 4   Age         58079 non-null  float64
 5   gender      58079 non-null  object 
 6   ethnicity   58079 non-null  object 
 7   insurance   58079 non-null  object 
 8   label       58079 non-null  int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 4.0+ MB


In [169]:
def split_df(dataframe):
    total_rows = len(dataframe)
    train_end = int(total_rows * 0.6)
    validation_end = train_end + int(total_rows * 0.2)

    dataframe['split'] = 'train'  # Default to 'train'
    dataframe.loc[train_end:validation_end-1, 'split'] = 'validation'
    dataframe.loc[validation_end:, 'split'] = 'test'
    return dataframe

cohort_new = cohort.replace({
    'WHITE - RUSSIAN': 'WHITE',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC OR LATINO',
    'ASIAN - CHINESE': 'ASIAN',
    'BLACK/CAPE VERDEAN': 'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - DOMINICAN': 'HISPANIC OR LATINO',
    'BLACK/CARIBBEAN ISLAND': 'BLACK/AFRICAN AMERICAN',
    'BLACK/AFRICAN': 'BLACK/AFRICAN AMERICAN',
    'PORTUGUESE': 'WHITE',
    'UNABLE TO OBTAIN': 'UNKNOWN',
    'WHITE - EASTERN EUROPEAN': 'WHITE',
    'ASIAN - SOUTH EAST ASIAN': 'ASIAN',                       
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC OR LATINO',
    'PATIENT DECLINED TO ANSWER': 'UNKNOWN',                     
    'AMERICAN INDIAN/ALASKA NATIVE': 'OTHER',                  
    'ASIAN - ASIAN INDIAN': 'ASIAN',                           
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC OR LATINO',
    'HISPANIC/LATINO - CUBAN': 'HISPANIC OR LATINO',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC OR LATINO',
    'HISPANIC/LATINO - HONDURAN': 'HISPANIC OR LATINO',
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC OR LATINO',
    'HISPANIC/LATINO - MEXICAN': 'HISPANIC OR LATINO',
    'WHITE - BRAZILIAN': 'WHITE',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'OTHER',       
    'SOUTH AMERICAN': 'HISPANIC OR LATINO',                                  
    'ASIAN - KOREAN': 'ASIAN',                                  
    'MULTIPLE RACE/ETHNICITY': 'OTHER',                         
})
cohort_new = cohort_new[(cohort_new.ethnicity=='WHITE')|(cohort_new.ethnicity=='BLACK/AFRICAN AMERICAN')]
cohort_new['insurance_01'] = cohort_new['insurance']=='Other'
split_dfs = []
for gender in ['F', 'M']:
    for label in [0, 1]:
        for ethnicity in ['WHITE', 'BLACK/AFRICAN AMERICAN']:
            for insurance_01 in [True, False]:
                subset = cohort_new[(cohort_new['gender'] == gender) & 
                                (cohort_new['label'] == label) & 
                                (cohort_new['ethnicity'] == ethnicity) & 
                                (cohort_new['insurance_01'] == insurance_01)]
                if not subset.empty:
                    split_dfs.append(split_df(subset.copy().reset_index()))
cohort_new = pd.concat(split_dfs).reset_index()                
cohort_new = cohort_new.drop(columns=['level_0', 'index', 'Age', 'insurance_01'])
print(cohort_new.split.value_counts())
cohort_new.sample(5)

train         30399
test          10148
validation    10129
Name: split, dtype: int64


,subject_id,hadm_id,admittime,dischtime,gender,ethnicity,insurance,label,split
42202,12494390.0,22959591.0,2152-01-09 20:53:00,2152-01-15 18:48:00,M,BLACK/AFRICAN AMERICAN,Other,0,train
9266,15064880.0,21603908.0,2176-05-11 16:36:00,2176-05-13 16:45:00,F,WHITE,Medicare,0,train
43874,13307398.0,20464684.0,2142-06-05 12:01:00,2142-06-07 18:00:00,M,BLACK/AFRICAN AMERICAN,Medicare,0,train
3472,18421025.0,21977383.0,2136-01-09 20:42:00,2136-02-01 15:30:00,F,WHITE,Other,0,test
64,10163823.0,22018958.0,2166-02-22 18:16:00,2166-03-02 17:30:00,F,WHITE,Other,0,train


In [170]:
cohort_new.to_csv(save_dir + 'cohort.csv')
cohort_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50676 entries, 0 to 50675
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  50676 non-null  float64
 1   hadm_id     50676 non-null  float64
 2   admittime   50676 non-null  object 
 3   dischtime   50676 non-null  object 
 4   gender      50676 non-null  object 
 5   ethnicity   50676 non-null  object 
 6   insurance   50676 non-null  object 
 7   label       50676 non-null  int64  
 8   split       50676 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.5+ MB


### Diag

In [171]:
diag = pd.read_csv(read_dir + 'preproc_diag.csv.gz', compression='gzip')
diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016267 entries, 0 to 1016266
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   subject_id    1016267 non-null  int64 
 1   hadm_id       1016267 non-null  int64 
 2   new_icd_code  1016267 non-null  object
dtypes: int64(2), object(1)
memory usage: 23.3+ MB


In [172]:
icd_map = pd.read_csv('./mimiciv/2.2/hosp/d_icd_diagnoses.csv.gz', compression='gzip')
icd_map = icd_map[icd_map.icd_version==10]

diag_new = pd.merge(diag, icd_map, left_on='new_icd_code', right_on='icd_code', how='left')
diag_new = diag_new[['subject_id','hadm_id','new_icd_code','long_title']]
diag_new = diag_new[diag_new.hadm_id.isin(cohort_new.hadm_id)]
diag_new.sample(5)

,subject_id,hadm_id,new_icd_code,long_title
944349,19281994,27832017,I34,Nonrheumatic mitral valve disorders
154533,11512649,24481953,M48,Other spondylopathies
788408,17746916,23419637,N23,Unspecified renal colic
16037,10172708,27836647,Z91,"Personal risk factors, not elsewhere classified"
982365,19667522,25933816,C91,Lymphoid leukemia


In [173]:
diag_new.to_csv(save_dir + 'diag.csv')
diag_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887627 entries, 0 to 1016266
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   subject_id    887627 non-null  int64 
 1   hadm_id       887627 non-null  int64 
 2   new_icd_code  887627 non-null  object
 3   long_title    887627 non-null  object
dtypes: int64(2), object(2)
memory usage: 33.9+ MB


### Labs

In [193]:
labs = pd.read_csv(read_dir + 'preproc_labs.csv.gz', compression='gzip')
labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14555160 entries, 0 to 14555159
Data columns (total 7 columns):
 #   Column               Dtype  
---  ------               -----  
 0   subject_id           int64  
 1   hadm_id              float64
 2   charttime            object 
 3   itemid               int64  
 4   admittime            object 
 5   lab_time_from_admit  object 
 6   valuenum             float64
dtypes: float64(2), int64(2), object(3)
memory usage: 777.3+ MB


In [194]:
lab_map = pd.read_csv('./mimiciv/2.2/hosp/d_labitems.csv.gz', compression='gzip')

labs_new = pd.merge(labs, lab_map, on='itemid', how='left')
labs_new = labs_new[['subject_id','hadm_id','itemid','lab_time_from_admit','valuenum','label','fluid','category']]
labs_new = labs_new[labs_new.hadm_id.isin(cohort_new.hadm_id)]
labs_new.sample(5)

,subject_id,hadm_id,itemid,lab_time_from_admit,valuenum,label,fluid,category
7579123,15179586,25277029.0,50947,2 days 10:33:00,0.00,I,Blood,Chemistry
8994065,16147477,28763964.0,50820,2 days 20:34:00,7.37,pH,Blood,Blood Gas
7021502,14787989,29506566.0,51450,0 days 14:47:00,34.00,Monos,Pleural,Hematology
11633892,17968573,28216346.0,51249,9 days 16:17:00,27.80,MCHC,Blood,Hematology
6532559,14422803,25576375.0,51277,0 days 18:45:00,16.90,RDW,Blood,Hematology


In [195]:
labs_new.to_csv(save_dir + 'labs.csv')
labs_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12408774 entries, 0 to 14555159
Data columns (total 8 columns):
 #   Column               Dtype  
---  ------               -----  
 0   subject_id           int64  
 1   hadm_id              float64
 2   itemid               int64  
 3   lab_time_from_admit  object 
 4   valuenum             float64
 5   label                object 
 6   fluid                object 
 7   category             object 
dtypes: float64(2), int64(2), object(4)
memory usage: 852.0+ MB


### Proc

In [198]:
proc = pd.read_csv(read_dir + 'preproc_proc.csv.gz', compression='gzip')
proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40698 entries, 0 to 40697
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   subject_id            40698 non-null  int64 
 1   hadm_id               40698 non-null  int64 
 2   icd_code              40698 non-null  object
 3   chartdate             40698 non-null  object
 4   admittime             40698 non-null  object
 5   proc_time_from_admit  40698 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [199]:
pcd_map = pd.read_csv('./mimiciv/2.2/hosp/d_icd_procedures.csv.gz', compression='gzip')
pcd_map = pcd_map[pcd_map.icd_version==10]

proc_new = pd.merge(proc, pcd_map, on='icd_code', how='left')
proc_new = proc_new[['subject_id','hadm_id','icd_code','proc_time_from_admit','long_title']]
proc_new = proc_new[proc_new.hadm_id.isin(cohort_new.hadm_id)]
proc_new.sample(5)

,subject_id,hadm_id,icd_code,proc_time_from_admit,long_title
29891,17358150,25270375,03CK0ZZ,7 days 00:00:00,Extirpation of Matter from Right Internal Caro...
37691,19266140,23162122,4B02XSZ,1 days 23:24:00,"Measurement of Cardiac Pacemaker, External App..."
6976,11651867,20236112,02HK3JZ,7 days 03:51:00,Insertion of Pacemaker Lead into Right Ventric...
28837,17074541,23192877,5A1955Z,1 days 16:38:00,"Respiratory Ventilation, Greater than 96 Conse..."
21487,15261807,29700270,02UJ0JZ,3 days 01:41:00,Supplement Tricuspid Valve with Synthetic Subs...


In [200]:
proc_new.to_csv(save_dir + 'proc.csv')
proc_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33549 entries, 0 to 40697
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   subject_id            33549 non-null  int64 
 1   hadm_id               33549 non-null  int64 
 2   icd_code              33549 non-null  object
 3   proc_time_from_admit  33549 non-null  object
 4   long_title            33549 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.5+ MB


### Meds

In [203]:
meds = pd.read_csv(read_dir + 'preproc_med.csv.gz', compression='gzip')
meds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1842600 entries, 0 to 1842599
Data columns (total 8 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   subject_id              int64 
 1   hadm_id                 int64 
 2   starttime               object
 3   stoptime                object
 4   drug_name               object
 5   start_hours_from_admit  object
 6   stop_hours_from_admit   object
 7   dose_val_rx             object
dtypes: int64(2), object(6)
memory usage: 112.5+ MB


In [204]:
meds_new = meds.copy()
meds_new = meds_new[['subject_id','hadm_id','drug_name','start_hours_from_admit','stop_hours_from_admit','dose_val_rx']]
meds_new = meds_new[meds_new.hadm_id.isin(cohort_new.hadm_id)]
meds_new.sample(5)

,subject_id,hadm_id,drug_name,start_hours_from_admit,stop_hours_from_admit,dose_val_rx
728159,19674244,26076868,sodium chloride,3 days 02:59:00,5 days 04:59:00,100
1428001,10109085,27429585,potassium chloride,1 days 13:30:00,1 days 12:30:00,20
1706122,17524541,23376619,amoxicillin and clavulanate potassium,1 days 15:27:00,1 days 20:27:00,500
411829,19843082,23323137,prednisone,12 days 13:48:00,14 days 12:48:00,40
1725661,10480035,26532273,hydroxyzine hydrochloride,3 days 08:29:00,4 days 07:29:00,12.5


In [205]:
meds_new.to_csv(save_dir + 'meds.csv')
meds_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567031 entries, 0 to 1842599
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   subject_id              1567031 non-null  int64 
 1   hadm_id                 1567031 non-null  int64 
 2   drug_name               1567031 non-null  object
 3   start_hours_from_admit  1567031 non-null  object
 4   stop_hours_from_admit   1567031 non-null  object
 5   dose_val_rx             1567031 non-null  object
dtypes: int64(2), object(4)
memory usage: 83.7+ MB


In [206]:
meds_new.drug_name.value_counts()

sodium chloride                    116461
furosemide                          95463
potassium chloride                  75341
dextrose monohydrate                73704
heparin sodium                      72999
                                    ...  
trimethobenzamide hydrochloride         1
levonorgestrel                          1
indigo carmine                          1
asenapine maleate                       1
sumatriptan                             1
Name: drug_name, Length: 905, dtype: int64